In [1]:
from moviepy.editor import *
from multiprocessing import Process, Semaphore
import sys
import pandas as pd
import xlrd
import soundfile as sf
import pyloudnorm as pyln

In [2]:
#function to read Video File

def VideoDescription(loc):
        try:
            clip= VideoFileClip(r'%s'%loc)
            w,h=clip.size
            print("Width: ",w," Height: ",h)
            print("Duration(mins): ",clip.duration/60)
            print("FPS: ", clip.fps)
        except:
            print("Video not found")
        else:
            return loc

        
#function to read csv

def ReadCSV(file):
        try:
            pd.read_excel(r'%s'%file)
            print("File Found")
            if not file.endswith(".xlsx"):
                raise FileNotFoundError

        except(FileNotFoundError):
            print("Error: File not found, Please Enter the correct path")
            
        else:
            return file

#function take output path
def VideoOutput(oloc):
        try:
            if not (os.path.exists(oloc)):
                raise FileNotFoundError

        except (FileNotFoundError, OSError, IOError):
            print("Not Valid ! Please check that you entered the correct path")
        else:
            return oloc

In [3]:
#function to split video

def write_videofile():
    try:
        for row in df.iterrows():
            clip_start = row["clip_start"]
            clip_end = row["clip_end"]
            
            hlclip = clip.subclip(clip_start, clip_end)
            print(clip_start, clip_end)
            hlclip.write_videofile("output_%s.mp4" % num)
            hlclip.audio.write_audio("output_%s.mp4"% num)
    except:
        print("error", clip_start, clip_end)
        

#function to split video files
def break_video_files(loc,file,oloc):
    global path
    num = 0
    loudness_dict = {}
    df = pd.read_excel(file)

    for ind in df.index:
        with VideoFileClip(loc) as clips:
           
            clip = clips.subclip(df['clip_start'][ind], df['clip_end'][ind])
            clip.write_videofile(os.path.join(oloc,"output_%s.mp4" % str(num).zfill(3)))
            clip.audio.write_audiofile(os.path.join(oloc,"output_%s.wav" % str(num).zfill(3)))
            num += 1

        for file_name in os.listdir(oloc):
            if file_name.endswith(".wav"):
                path = os.path.join(oloc, file_name)
                data, rate = sf.read(path)  
                meter = pyln.Meter(rate)  
                loudness = meter.integrated_loudness(data)
                file_name=file_name.split(".")[0] + ".mp4"
                loudness_dict[file_name] = loudness
    return loudness_dict,path

In [4]:
#function to merge clips
def merge_files(loudness_dict):
    sorted_wav_list = sorted(loudness_dict.items(),key=lambda kv: (kv[1], kv[0]))
    
    sorted_wav_list= sorted_wav_list[:15]
    print(sorted_wav_list)

    new_wav_list = []
    
    for loud_file in sorted_wav_list:
        new_wav_list.append(loud_file[0])
    new_wav_list = sorted(new_wav_list)
    print(new_wav_list)

    new_video_list = []
    for i in new_wav_list:
        new_video_list.append(VideoFileClip(os.path.join(oloc, i)))
    
    final_clip = concatenate_videoclips(new_video_list)
    
    final_clip.write_videofile(os.path.join(oloc, "final_highlights.mp4"))
    floc = os.path.abspath(oloc)
    clip_path = os.path.dirname(floc)
    
    if os.path.isdir(clip_path):
        print("Your highlights video is located in {0} with name {1} ".format(clip_path, os.path.basename(floc)))
        
    return final_clip

#function to remove the generated output.mp4 and output.wav files from our directory
def clean_files(oloc):
    
    dir_files = os.listdir(oloc)
    for dir_file in dir_files:
    
        if dir_file.startswith("output"):
            os.remove(os.path.join(oloc, dir_file))

In [5]:
loc=VideoDescription(input("Enter your Video File Location: "))

Enter your Video File Location: C:\Users\samee\Documents\Courses\Projects\Lawn Tennis Highlights with Python\LTmatch.mp4
Width:  1280  Height:  720
Duration(mins):  94.21683333333334
FPS:  30.0


In [6]:
file=ReadCSV(input("Enter CLIP TIME csv location: "))


Enter CLIP TIME csv location: C:\Users\samee\Documents\Courses\Projects\Lawn Tennis Highlights with Python\LTHL.xlsx
File Found


In [7]:
oloc = VideoOutput(input("Enter output Folder path?"))

Enter output Folder path?C:\Users\samee\Documents\Courses\Projects\Lawn Tennis Highlights with Python\Output


In [8]:
 loudness_dict, path = break_video_files(loc, file,oloc)

chunk:   1%|▌                                                                | 2/221 [00:00<00:14, 15.62it/s, now=None]

Moviepy - Building video C:\Users\samee\Documents\Courses\Projects\Lawn Tennis Highlights with Python\Output\output_000.mp4.
MoviePy - Writing audio in %s


t:   3%|█▊                                                                   | 8/300 [00:00<00:03, 79.20it/s, now=None]

MoviePy - Done.
Moviepy - Writing video C:\Users\samee\Documents\Courses\Projects\Lawn Tennis Highlights with Python\Output\output_000.mp4



chunk:   1%|▌                                                                | 2/221 [00:00<00:14, 14.81it/s, now=None]

...

t:   2%|█▏                                                                   | 7/390 [00:00<00:05, 67.30it/s, now=None]

MoviePy - Done.
Moviepy - Writing video C:\Users\samee\Documents\Courses\Projects\Lawn Tennis Highlights with Python\Output\output_019.mp4



chunk:   1%|▍                                                                | 2/287 [00:00<00:18, 15.15it/s, now=None]

Moviepy - Done !
Moviepy - video ready C:\Users\samee\Documents\Courses\Projects\Lawn Tennis Highlights with Python\Output\output_019.mp4
MoviePy - Writing audio in %s


MoviePy - Done.


In [9]:
final_clip = merge_files(loudness_dict)

[('output_012.mp4', -38.389891126940825), ('output_019.mp4', -37.798238374476085), ('output_015.mp4', -37.407434150482494), ('output_000.mp4', -36.89905576133344), ('output_009.mp4', -36.834357275971946), ('output_002.mp4', -36.09360167249582), ('output_007.mp4', -35.97849644626495), ('output_008.mp4', -35.40590801911571), ('output_001.mp4', -34.068272027595945), ('output_017.mp4', -34.04739529694406), ('output_011.mp4', -34.00399229071446), ('output_014.mp4', -33.606531671419816), ('output_003.mp4', -32.79698677324668), ('output_018.mp4', -32.751816852492645), ('output_006.mp4', -32.74044135682667)]
['output_000.mp4', 'output_001.mp4', 'output_002.mp4', 'output_003.mp4', 'output_006.mp4', 'output_007.mp4', 'output_008.mp4', 'output_009.mp4', 'output_011.mp4', 'output_012.mp4', 'output_014.mp4', 'output_015.mp4', 'output_017.mp4', 'output_018.mp4', 'output_019.mp4']


t:   0%|                                                                    | 6/3930 [00:00<01:06, 58.82it/s, now=None]

Moviepy - Building video C:\Users\samee\Documents\Courses\Projects\Lawn Tennis Highlights with Python\Output\final_highlights.mp4.
Moviepy - Writing video C:\Users\samee\Documents\Courses\Projects\Lawn Tennis Highlights with Python\Output\final_highlights.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\samee\Documents\Courses\Projects\Lawn Tennis Highlights with Python\Output\final_highlights.mp4
Your highlights video is located in C:\Users\samee\Documents\Courses\Projects\Lawn Tennis Highlights with Python with name Output 


In [ ]:
clean_files(oloc)